## Задача

Необходимо привести данные в читаемый и понятный для работы вид

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.stats import shapiro
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

## Изучение данных

In [2]:
data = pd.read_csv('/datasets/data_cleaning_challenge.csv')
data

,Row Type,Iter Number,Power1,Speed1,Speed2,Electricity,Effort,Weight,Torque,Unnamed: 9,Unnamed: 10
0,first name: Person,last name: Human,date: end of time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Row Type,Iter Number,Power1,Speed1,Speed2,Electricity,Effort,Weight,Torque,NaN,NaN
3,Iter,1,360,108,863,599,680,442,982,NaN,NaN
4,Iter,2,684,508,613,241,249,758,639,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
76372,Iter,6,879,73,977,680,500,395,863,NaN,NaN
76373,Average,979,641,531,374,448,407,185,439,NaN,NaN
76374,Maximum,783,172,941,53,982,217,963,502,NaN,NaN
76375,Std.Dev.,221,112,717,630,239,561,142,909,NaN,NaN


## Обработка данных

In [3]:
# уберу пустые столбцы
data = data.drop(columns=['Unnamed: 9', 'Unnamed: 10'])

In [4]:
data

,Row Type,Iter Number,Power1,Speed1,Speed2,Electricity,Effort,Weight,Torque
0,first name: Person,last name: Human,date: end of time,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Row Type,Iter Number,Power1,Speed1,Speed2,Electricity,Effort,Weight,Torque
3,Iter,1,360,108,863,599,680,442,982
4,Iter,2,684,508,613,241,249,758,639
...,...,...,...,...,...,...,...,...,...
76372,Iter,6,879,73,977,680,500,395,863
76373,Average,979,641,531,374,448,407,185,439
76374,Maximum,783,172,941,53,982,217,963,502
76375,Std.Dev.,221,112,717,630,239,561,142,909


In [5]:
#уберем пропущенные строки
data_no_nans = data.copy()
data_no_nans = data_no_nans[data['Row Type'].notna()].reset_index()

In [6]:
data_no_nans.drop(columns='index', inplace=True)

In [7]:
# создадим лист, в котором для каждого пользователя присвоим порядковый номер
person_number = []
counter = 0

for i in data_no_nans['Row Type']:
    if 'first name' in i:
        counter += 1
    person_number.append(counter)

In [8]:
#person_number

In [9]:
# добавим лист как новый столбец
data_no_nans['person_number'] = person_number
data_no_nans

,Row Type,Iter Number,Power1,Speed1,Speed2,Electricity,Effort,Weight,Torque,person_number
0,first name: Person,last name: Human,date: end of time,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Row Type,Iter Number,Power1,Speed1,Speed2,Electricity,Effort,Weight,Torque,1
2,Iter,1,360,108,863,599,680,442,982,1
3,Iter,2,684,508,613,241,249,758,639,1
4,Iter,3,365,126,825,407,855,164,86,1
...,...,...,...,...,...,...,...,...,...,...
58392,Iter,6,879,73,977,680,500,395,863,5994
58393,Average,979,641,531,374,448,407,185,439,5994
58394,Maximum,783,172,941,53,982,217,963,502,5994
58395,Std.Dev.,221,112,717,630,239,561,142,909,5994


In [10]:
# уберу строки, повторяющие название столбцов
data_no_nans = data_no_nans[data_no_nans['Row Type'] != 'Row Type']

In [11]:
data_no_nans

,Row Type,Iter Number,Power1,Speed1,Speed2,Electricity,Effort,Weight,Torque,person_number
0,first name: Person,last name: Human,date: end of time,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Iter,1,360,108,863,599,680,442,982,1
3,Iter,2,684,508,613,241,249,758,639,1
4,Iter,3,365,126,825,407,855,164,86,1
5,Iter,4,764,594,304,718,278,674,774,1
...,...,...,...,...,...,...,...,...,...,...
58392,Iter,6,879,73,977,680,500,395,863,5994
58393,Average,979,641,531,374,448,407,185,439,5994
58394,Maximum,783,172,941,53,982,217,963,502,5994
58395,Std.Dev.,221,112,717,630,239,561,142,909,5994


In [12]:
# выделю имя и фамилию каждого человека в ДФ-е
df_by_name = data_no_nans[data_no_nans['Row Type'].str.contains('first name')]

# уберу лишние столбы
df_by_name.drop(columns=['Speed1', 'Speed2', 'Electricity', 'Effort', 'Weight', 'Torque'], inplace=True)

# переименую столбы
df_by_name.columns = ['First name', 'Last name', 'Date', 'person_number']

# оставлю только данные, не дублирующие название столбцов
df_by_name['First name'] = df_by_name['First name'].str[12:]
df_by_name['Last name'] = df_by_name['Last name'].str[11:]
df_by_name['Date'] = df_by_name['Date'].str[6:]

In [13]:
df_by_name

,First name,Last name,Date,person_number
0,Person,Human,end of time,1
11,Person,Human,end of time,2
22,Person,Human,end of time,3
33,Person,Human,end of time,4
44,Person,Human,end of time,5
...,...,...,...,...
58342,Person,Human,end of time,5990
58352,Person,Human,end of time,5991
58363,Person,Human,end of time,5992
58374,Person,Human,end of time,5993


In [14]:
# создад ДФ без имен и фамилий
df_no_name = data_no_nans[~data_no_nans['Row Type'].str.contains('first name')]

In [15]:
df_no_name

,Row Type,Iter Number,Power1,Speed1,Speed2,Electricity,Effort,Weight,Torque,person_number
2,Iter,1,360,108,863,599,680,442,982,1
3,Iter,2,684,508,613,241,249,758,639,1
4,Iter,3,365,126,825,407,855,164,86,1
5,Iter,4,764,594,304,718,278,674,774,1
6,Iter,5,487,97,593,206,779,800,123,1
...,...,...,...,...,...,...,...,...,...,...
58392,Iter,6,879,73,977,680,500,395,863,5994
58393,Average,979,641,531,374,448,407,185,439,5994
58394,Maximum,783,172,941,53,982,217,963,502,5994
58395,Std.Dev.,221,112,717,630,239,561,142,909,5994


In [16]:
final_data = pd.merge(left=df_by_name, right=df_no_name, on='person_number')
final_data.drop(columns='person_number', inplace=True)

In [17]:
final_data.head(15)

,First name,Last name,Date,Row Type,Iter Number,Power1,Speed1,Speed2,Electricity,Effort,Weight,Torque
0,Person,Human,end of time,Iter,1,360,108,863,599,680,442,982
1,Person,Human,end of time,Iter,2,684,508,613,241,249,758,639
2,Person,Human,end of time,Iter,3,365,126,825,407,855,164,86
3,Person,Human,end of time,Iter,4,764,594,304,718,278,674,774
4,Person,Human,end of time,Iter,5,487,97,593,206,779,800,123
5,Person,Human,end of time,Average,182,361,741,231,731,493,847,237
6,Person,Human,end of time,Maximum,276,33,97,154,25,922,9,312
7,Person,Human,end of time,Std.Dev.,523,1000,34,904,237,600,170,553
8,Person,Human,end of time,Total,336,-,-,-,-,977,744,652
9,Person,Human,end of time,Iter,1,702,494,311,492,456,370,150
